In [ ]:
%pip install sentence-transformers transformers accelerate scikit-learn -q
%pip install regex


In [ ]:
import re
from typing import List, Dict
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:

SECTION_HEADERS = {
    "summary": ["summary", "professional summary", "profile", "Highlights", "Executive Summary"],
    "work_history": ["experience", "employment", "work history", "professional experience"],
    "education": ["education", "academic background"],
    "skills": ["skills", "technical skills"],
    "certifications": ["certifications", "licenses"],
    "projects": ["projects"],
    "others": ["languages", "awards", "publications", "interests"]
}

SECTION_LABELS = {
    "summary": "Candidate summary or profile",
    "work_history": "Jobs, roles, experience",
    "education": "Degrees and academic background",
    "skills": "Technical and soft skills",
    "certifications": "Certifications and licenses",
    "projects": "Project experience",
    "others": "Languages, awards, publications"
}


In [ ]:

embedder = SentenceTransformer("all-MiniLM-L6-v2")
label_embeddings = {label: embedder.encode(desc) for label, desc in SECTION_LABELS.items()}

In [ ]:
import regex as re

def normalize_resume_lines(text: str) -> str:
    # Clean common formatting noise
    text = text.replace('\\/', '/').replace('\\t', ' ').replace('\\n', '\n')

    # Collapse tabs and unify repeated spacing
    text = re.sub(r'\t+', ' ', text)
    text = re.sub(r' {3,}', '\n', text)  # Break on 3+ spaces (common section divider)

    # Normalize spacing around newlines
    text = re.sub(r' *\n *', '\n', text)
    text = re.sub(r'\n{2,}', '\n', text)

    # Remove trailing space
    return text.strip()


In [ ]:
def clean_resume_text(text: str) -> str:
    text = re.sub(r'[ \t]+', ' ', text)
    text = re.sub(r'\n{2,}', '\n\n', text)
    return text.strip()

In [ ]:
def detect_section_blocks(text: str) -> List[Dict[str, str]]:
    lines = [line.strip() for line in text.splitlines() if line.strip()]
    blocks = []
    pending_header = None
    current_content = []
    current_label = None
    contact_info_lines = []
    found_section = False

    def classify_block(content: str) -> str:
        para_embedding = embedder.encode(content)
        sims = {
            label: cosine_similarity([para_embedding], [label_embeddings[label]])[0][0]
            for label in SECTION_LABELS
        }
        best_label, score = max(sims.items(), key=lambda x: x[1])
        return best_label if score >= 0.6 else "unknown"

    def add_block(content_lines: List[str], label: str = None):
        full_content = "\n".join(content_lines).strip()
        if not full_content:
            return
        #section_type = label or classify_block(full_content)
        # Use label if given; otherwise classify
        if label:
            section_type = label
        else:
            section_type = classify_block(full_content)
            # Fallback: long but unrecognized blocks → summary
            if section_type == "unknown" and len(full_content) > 150:
                section_type = "summary"
        blocks.append({"chunk": full_content, "section_type": section_type})

    for line in lines:
        line_lower = line.lower()

        # Try to match header
        matched_label = None
        for label, patterns in SECTION_HEADERS.items():
            if any(p == line_lower for p in patterns):
                matched_label = label
                break

        if matched_label:
            if pending_header and current_content:
                add_block([pending_header] + current_content, current_label)
                current_content = []
            elif pending_header:
                add_block([pending_header], current_label)

            pending_header = line
            current_label = matched_label
            found_section = True
        else:
            if not found_section:
                contact_info_lines.append(line)
            else:
                current_content.append(line)

    # Final flush
    if pending_header and current_content:
        add_block([pending_header] + current_content, current_label)
    elif pending_header:
        add_block([pending_header], current_label)
    elif current_content:
        add_block(current_content, current_label)

    if contact_info_lines:
        # 🧠 Split first line (e.g., name/title/headline) if it looks like a short contact label
        first_line = contact_info_lines[0].strip()
        rest_lines = contact_info_lines[1:]

        # If short enough, treat the first line as contact_info
        if len(first_line) <= 60:
            blocks.append({"chunk": first_line, "section_type": "contact_info"})

        if rest_lines:
            full_rest = "\n".join(rest_lines).strip()
            if full_rest:
                rest_label = classify_block(full_rest)
                blocks.append({"chunk": full_rest, "section_type": rest_label if rest_label != "unknown" else "summary"})



    return blocks


In [ ]:
import re
from typing import List, Dict

def postprocess_chunks(chunks: List[Dict[str, str]]) -> List[Dict[str, str]]:
    refined_chunks = []

    # Step 1: Split large work_history chunks into individual job roles
    for chunk in chunks:
        if chunk["section_type"] == "work_history" and len(chunk["chunk"]) > 2000:
            # Split on lines like "05/2014 - Current" or "09/2009 - 05/2014"
            entries = re.split(r"\n(?=\d{2}/\d{4}\s*[-–]\s*\d{2}/\d{4})", chunk["chunk"])
            for entry in entries:
                clean_entry = entry.strip()
                if len(clean_entry) > 100:
                    refined_chunks.append({
                        "chunk": clean_entry,
                        "section_type": "work_history"
                    })
        else:
            refined_chunks.append(chunk)

    # Step 2: Merge very small chunks that are not contact_info
    merged_chunks = []
    buffer = None

    for chunk in refined_chunks:
        chunk_len = len(chunk["chunk"])
        section = chunk["section_type"]

        if section != "contact_info" and chunk_len < 150:
            if buffer:
                buffer["chunk"] += "\n" + chunk["chunk"]
            else:
                buffer = chunk
        else:
            if buffer:
                merged_chunks.append(buffer)
                buffer = None
            merged_chunks.append(chunk)

    if buffer:
        merged_chunks.append(buffer)

    return merged_chunks


In [ ]:
# ✅ Main entry function
def chunk_resume_with_nlp(text: str) -> List[Dict[str, str]]:
    text = normalize_resume_lines(text)
    text = clean_resume_text(text)
    chunks =  detect_section_blocks(text)
    final_chunks = postprocess_chunks(chunks)
    return final_chunks

In [ ]:
# ✅ Optional test
def test_chunking(resume_text: str):
    print("🔍 Testing NLP-based resume chunking...\n")
    chunks = chunk_resume_with_nlp(resume_text)
    for idx, chunk in enumerate(chunks):
        print(f"--- Chunk {idx+1} ({chunk['section_type']}) chunk size {len(chunk["chunk"])} ---")
        print(chunk['chunk'])
        print()
    return chunks

In [ ]:
# Sample run
sample_resume = """
John Doe

SUMMARY
Experienced software engineer with 10+ years...

PROFESSIONAL EXPERIENCE
Senior Software Engineer, ABC Corp
Built microservices in Java and deployed on Kubernetes.

EDUCATION
MS in Computer Science, XYZ University

SKILLS
Java, React, Spring Boot, SQL, Docker
"""
test_chunking(sample_resume)

In [ ]:
sample_resume = r"""         ACCOUNTANT         Highlights        Microsoft Office :\tIntermediate in all the Microsoft Office components( Excel, Word, PowerPoint, Outlook and Access); Very familiar with ( Macros, V-look ups, SQL, calculating formulas and manipulating reports as well as smart view) Running query reports and creating reports.  SAP PeopleSoft:          Navigate within Resource one to retrieve financial reports and run queries              Experience     08\/2014   to   Current     Accountant    Company Name   －   City  ,   State      An accountant within the Experimental Therapeutics Research department, which prepares grant, restricted and non-restricted financial reports to  PI's within the department.  Create financial models and analysis for all restricted and non-restricted accounts within our department.  Prepares monthly financial reports for PI's.  Reconcile the ncRNA Core on a monthly basis.  Assists with the completion of effort reporting as well as completes Job Data Update Form ( JDUF) via the EPAF system and expense transfers as needed to clear deficit.  Assist with the preparation of operating budget.  Assist with financial model for specific grants within our department.  Monitors and reviews departmental transactions to ensure compliance with established financial controls in accordance with divisional and institutional policies and to ensure applicable revenue and expenses are captured.  Communicates with Grants and Contracts to ensure that requests completed in a timely manner.  Reviews Open PO on a monthly basis and close all PO's as necessary.  Analyzes and evaluates operations of financial systems, prepares recommendations and documents to update fiscal services and other related policies and procedures with respect to operating systems.  Assist with the invoicing via ILABS.  Construct a macro-enabled reconciliation template which significantly reduces keying errors while maximize reconciliation efficiency.  Implemented a macro-enabled reconciliation template, which increased effectiveness by twenty percent and significantly reduced keying errors.  Prepares training materials within the department to increase productivity and standardized processes.  Completes special projects as assigned by the department Administrator.         12\/2012   to   08\/2014     Staff Accountant    Company Name   －   City  ,   State      Full cycle accountant for several operating companies within Sysco Foods Company; completed and issued weekly, monthly, and quarterly financial statements to the operating companies for review and submit to the corporate office.  Uplaod and run reports in business objects for CFO ofoperating company to review.  Verified journal ledger entries of cash and check payments, purchases, expenses and trial balances by examining and authenticating inventory items.  Prepared monthly P&L for several operating companies within the Sysco Corporation and research variances, reviewed transactions in the GL for accuracy, and researched transactions that deviate from the purpose of the account.  Reconciled aging AR transactions and performed financial analysis, identified and explained deviations from planned or historical data.  Assisted in evaluating control systems in the accounting process to ensure operating companies comply with GAAP as well as provided the necessary information to both internal and external auditors on an as needed basis.  Reviewed account reconciliation from other accountants in a timely manner.         01\/2010   to   02\/2012     Accountant \/Auditor    Company Name   －   City  ,   State      Served as both an accountant and auditor within the department of financial services.  The role consisted of utilizing software applications to compile, retrieve, and summarize accounting information for analysis and reporting purposes.  Examined accounting source documents for accuracy, completeness, and compliance with departmental and state rules, regulations, and agreements.  Provided financial trainings for over 240 faculty and staff employees to enforce the rules and regulations of the procurement card set by set the State of Texas procurement regulation.  Maintained controlling records required to ensure accuracy of all data entered into the accounting system.  Identified and reported situations not in compliance with PVAMU internal controls, policies, and procedures, recommending improvements to the accounting process to optimize internal control.  Performed duties in compliance with GAAP, company and department policies and procedures, internal controls and Sarbanes-Oxley requirements.  Reviewed expense report for irregularities and recommended corrective measures to improve internal controls.  Prepared adjusting and closing entries, statements, and analyze financial operations.          Education     May 2011     Masters of Science  :   Accounting    Prairie View A&M University   －   City  ,   State      Accounting       Dec 2009     BBA  :   Accounting    Prairie View A&M University   －   City  ,   State      Accounting        Professional Affiliations    CPA Candidate Participated in various community activities, including the Phi Beta Lambda and  National     Association of Black Accountants Texas Society of CPA's Beta Gamma Sigma Greater Women's Chamber of Society Completed all the Research Operations Curriculum in 2015       Skills    account reconciliation, accounting, accountant, accounting system, AR, budget, business objects, closing, Contracts, financial, financial analysis, financial controls, financial operations, financial reports, financial statements, GL, Grants, inventory, invoicing, ledger, Macros, Access, Excel, Microsoft Office, office, Outlook, PowerPoint, Word, Monitors, operating systems, PeopleSoft, policies, processes, procurement, reporting, Research, SAP, Sarbanes-Oxley, training materials, view   """
test_chunking(sample_resume)

In [ ]:
sample_resume = r"""         AUTOMOBILE TRANSPORTER       Professional Summary     Dedicated, responsible Class A driver  with a clean driver's license. Self-motivated and customer-focused.Exceptional leader talented at leadership, communication, and management skills. Excellent team player with over 20 years of experience in the military and trucking industry.Talented Mechanist Tech with more than 8 years of experience developing and executing maintenance programs. Excellent troubleshooting skills. Areas of expertise include mechanics, technology, and management.          Key Skills          Quality control  OSHA regulatory compliance  Dependable independent worker  Clean D.O.T.  Good at following instructions      Solid communication skills  Strong organizational skills  Strong work ethic  Team player  Timely project completion            Work Experience     05\/2014   -   Current     Company Name   －   City  ,   State    Automobile Transporter        Conducted daily DOT pre-trip inspections according to a set checklist.  Hooked and unhooked trailers from the tractor.  Coordinated daily delivery schedules based on customer schedules, peak delivery times and alternate routes.  Perform emergency repairs including service hydraulic system.  Maintain accurate records such as vehicle logs, records of cargo and billing statements in accordance with regulations.  Obeyed traffic laws and followed established traffic and transportation procedures  Transported freight from origin to destination in a safe and timely manner.         09\/2009   -   05\/2014     Company Name   －   City  ,   State    Driver\/Car Hauler        Coordinated daily delivery schedules based on customer schedules, peak delivery times and alternate routes.  Conducted daily DOT pre-trip inspections according to a set checklist.  Maintained records required for compliance with state and federal regulations.  Interacted with customers and vendors in a friendly and timely manner.  Transported freight from origin to destination in a safe and timely manner.  Checked load accuracy and stability before each trip.  Picked up customer loads in a timely and accurate manner.  Maintained telephone and radio contact with supervisor to receive delivery instructions.  Conducted emergency roadside repairs, including changing tires, replacing light bulbs and installing fuses and tire chains.  Recorded expenses and maintained receipts.         02\/2006   -   09\/2009     Company Name   －   City  ,   State    Independent Contractor         Facilitated successful internal and external audits through sound and thorough documentation.  Composed effective accounting reports summarizing accounts payable data.  Posted receipts to appropriate general ledger accounts.  Reconciled discrepancies between accounts receivable general ledger account and accounts receivable trial balance account.  Balanced monthly general ledger accounts to accurately record cost and month end accruals.  Supervised invoice processing, purchase orders, expense reports, credit memos and payment transactions.  Process accounts payable for 3 employees.  Entered financial data into the company accounting database to be verified and reconciled.  Tracked financial progress by creating quarterly and yearly balance sheets.  Streamlined bookkeeping procedures to increase efficiency and productivity.  Developed monthly, quarterly and annual profit and loss statements and balance sheets.  Performed periodic budgeting\/modeling to project monthly cash requirements.  Drove car transporter, applying knowledge of commercial driving regulations in order to assure prompt delivery to dealerships.  Verified the contents of inventory to match bill of lading forms.  Maintain driver log.         10\/2004   -   02\/2006     Company Name   －   City  ,   State    Driver\/Car Hauler        Transport automobiles from manufacturing plant to the dealership.  Worker directly with training, departments, co-signees, and management to achieve on prompt deliveries new and pre-owned vehicles.  Perform emergency repairs including service hydraulic system.  Maintain accurate records such as vehicle logs, records of cargo and billing statements in accordance with regulations.  Obeyed traffic laws and followed established traffic and transportation procedures.  Trained New drivers.         08\/2003   -   10\/2004     Company Name   －   City  ,   State    Driver\/Car Hauler        Coordinated daily delivery schedules based on customer schedules, peak delivery times and alternate routes.  Obtained and maintained proper delivery authorization and pickup documentation.  Conducted daily DOT pre-trip inspections according to a set checklist.  Loaded and unloaded trailers with mechanical freight handling equipment.  Delivered personal Vehicles to homes and places of business within established time frames.  Interacted with customers and vendors in a friendly and timely manner.         09\/2000   -   05\/2003     Company Name   －   City  ,   State    Driver        Safely loaded and unloaded deliveries according to size of load and content description.  Transported deliveries locally and over the road.  Managed discrepancy documentation for incoming shipments.  Submitted all delivery documentation in a timely manner.  Maintained a daily, legible DOT log book and submitted corresponding documents.  Completed basic maintenance such as minor repairs to keep vehicle neat and running properly.  Reported all accidents, damage and malfunctions involving company equipment to management.  Inspected the truck for defects and safe operating condition before, during and after trips.  Established and maintained excellent customer relationships.  Answered customer questions regarding delivery promptly and accurately.  Loaded and unloaded freight to assure safety and minimize risk of damage and dangerous conditions.         04\/1992   -   09\/2000     Company Name   －   City  ,   State    Aviation Machinist Mate        Supervised the creation and validation of anti-terrorism plans on all ships.  Developed and led training programs in preparation for combat.  Operated and maintained communications equipment.  Documented and processed classified materials.  Recommended retransmission equipment employment based on capabilities of the equipment and the mission requirements.  Guided and coordinated unit's force protection programs to meet fleet requirements.  Planned and briefed personnel on missions.  Established and commanded field communication operations.  Routinely checked measuring equipment to resolve testing problems.  Monitored production operations for compliance with specifications and promptly reported defects.  Discarded and rejected products, materials and equipment that did not meet specifications.  Worked within flight operations to maintain a culture of safety with efficient procedures.  Prepared and filed flight plans both domestic and international, utilizing all available resources and aircraft capabilities for maximum efficiency.  Ensured that all aircraft operational matters were within manufacturers, FAA and company limits including fuel loading, weight and balance and performance measures.  Coordinated communication between air traffic control and maintenance personnel.  Performed and supervised airfield management activities.  Monitored gauges, warning devices and control panels to verify aircraft performance and to regulate engine speed.  Calculated the aircraft weight and balance, performance and fuel requirements.  Assisted with the Safety Risk Management (SRM) process.  Announced and demonstrated safety and emergency procedures.          Specialized Training     2000     Road Master Truck driving  School   －   City  ,   State  ,   United States      Truck Driving    GPA:   GPA: 3.7     Class A CDL License             Road Master Truck Driving School  City  ,   State      Class \"A\" Commercial Driver License (CDL)        Credentials     TWIC card    """
test_chunking(sample_resume)

In [1]:
# Utility to preview output
def test_normalization(text: str):
    print("🧪 Normalized Resume Lines:\n")
    normalized = normalize_resume_lines(text)
    for line in normalized.splitlines():
        print(line)
    return normalized

In [ ]:
sample_resume = r"""         ACCOUNTANT         Highlights        Microsoft Office :\tIntermediate in all the Microsoft Office components( Excel, Word, PowerPoint, Outlook and Access); Very familiar with ( Macros, V-look ups, SQL, calculating formulas and manipulating reports as well as smart view) Running query reports and creating reports.  SAP PeopleSoft:          Navigate within Resource one to retrieve financial reports and run queries              Experience     08\/2014   to   Current     Accountant    Company Name   －   City  ,   State      An accountant within the Experimental Therapeutics Research department, which prepares grant, restricted and non-restricted financial reports to  PI's within the department.  Create financial models and analysis for all restricted and non-restricted accounts within our department.  Prepares monthly financial reports for PI's.  Reconcile the ncRNA Core on a monthly basis.  Assists with the completion of effort reporting as well as completes Job Data Update Form ( JDUF) via the EPAF system and expense transfers as needed to clear deficit.  Assist with the preparation of operating budget.  Assist with financial model for specific grants within our department.  Monitors and reviews departmental transactions to ensure compliance with established financial controls in accordance with divisional and institutional policies and to ensure applicable revenue and expenses are captured.  Communicates with Grants and Contracts to ensure that requests completed in a timely manner.  Reviews Open PO on a monthly basis and close all PO's as necessary.  Analyzes and evaluates operations of financial systems, prepares recommendations and documents to update fiscal services and other related policies and procedures with respect to operating systems.  Assist with the invoicing via ILABS.  Construct a macro-enabled reconciliation template which significantly reduces keying errors while maximize reconciliation efficiency.  Implemented a macro-enabled reconciliation template, which increased effectiveness by twenty percent and significantly reduced keying errors.  Prepares training materials within the department to increase productivity and standardized processes.  Completes special projects as assigned by the department Administrator.         12\/2012   to   08\/2014     Staff Accountant    Company Name   －   City  ,   State      Full cycle accountant for several operating companies within Sysco Foods Company; completed and issued weekly, monthly, and quarterly financial statements to the operating companies for review and submit to the corporate office.  Uplaod and run reports in business objects for CFO ofoperating company to review.  Verified journal ledger entries of cash and check payments, purchases, expenses and trial balances by examining and authenticating inventory items.  Prepared monthly P&L for several operating companies within the Sysco Corporation and research variances, reviewed transactions in the GL for accuracy, and researched transactions that deviate from the purpose of the account.  Reconciled aging AR transactions and performed financial analysis, identified and explained deviations from planned or historical data.  Assisted in evaluating control systems in the accounting process to ensure operating companies comply with GAAP as well as provided the necessary information to both internal and external auditors on an as needed basis.  Reviewed account reconciliation from other accountants in a timely manner.         01\/2010   to   02\/2012     Accountant \/Auditor    Company Name   －   City  ,   State      Served as both an accountant and auditor within the department of financial services.  The role consisted of utilizing software applications to compile, retrieve, and summarize accounting information for analysis and reporting purposes.  Examined accounting source documents for accuracy, completeness, and compliance with departmental and state rules, regulations, and agreements.  Provided financial trainings for over 240 faculty and staff employees to enforce the rules and regulations of the procurement card set by set the State of Texas procurement regulation.  Maintained controlling records required to ensure accuracy of all data entered into the accounting system.  Identified and reported situations not in compliance with PVAMU internal controls, policies, and procedures, recommending improvements to the accounting process to optimize internal control.  Performed duties in compliance with GAAP, company and department policies and procedures, internal controls and Sarbanes-Oxley requirements.  Reviewed expense report for irregularities and recommended corrective measures to improve internal controls.  Prepared adjusting and closing entries, statements, and analyze financial operations.          Education     May 2011     Masters of Science  :   Accounting    Prairie View A&M University   －   City  ,   State      Accounting       Dec 2009     BBA  :   Accounting    Prairie View A&M University   －   City  ,   State      Accounting        Professional Affiliations    CPA Candidate Participated in various community activities, including the Phi Beta Lambda and  National     Association of Black Accountants Texas Society of CPA's Beta Gamma Sigma Greater Women's Chamber of Society Completed all the Research Operations Curriculum in 2015       Skills    account reconciliation, accounting, accountant, accounting system, AR, budget, business objects, closing, Contracts, financial, financial analysis, financial controls, financial operations, financial reports, financial statements, GL, Grants, inventory, invoicing, ledger, Macros, Access, Excel, Microsoft Office, office, Outlook, PowerPoint, Word, Monitors, operating systems, PeopleSoft, policies, processes, procurement, reporting, Research, SAP, Sarbanes-Oxley, training materials, view   """
test_normalization(sample_resume)

In [ ]:
sample_resume = r"""         ADVOCATE       Summary    In-depth knowledge of taking orders, compute charges, and administer billing or payments.\n* Hands-on experience in reviewing, updating or making changes to customer accounts.\n* Track record of listening and responding to customers' needs and concerns.\n* Able to handle returns and complaints effectively.\n* Demonstrated ability to record details of client contacts and service history.\n* Proven ability to determine answers and solutions quickly.\n* Special talent for handling irate and angry customers.\n* Proven written and oral communication skills\n* Proficient in managing business correspondence\n* Ability to organize personal work priorities\n* Knowledge of filing and updating records\n* Ability to work independently and as part of a team\n* Excellent organizational skills\n* Expert in handling office equipment\n* Internet savvy with a proven expertise in using MS Office applications\n* Excellent customer service orientation\n* Special talent for researching and analyzing data effectively\n* Exceptional attention to detail with proven interpersonal skills      Skills          Microsoft Word  Microsoft Office  Microsoft Outlook  Microsoft Power Point  Microsoft Excel  Mostly Microsoft Excel and Microsoft Word                Experience     10\/2016   to   Current     Advocate    Company Name   －   City  ,   State     Respond to and resolve on the first call, customer service inquires and issues by identifying the topic and type of assistance the caller needs such as benefits, eligibility and claims, financial spending accounts and correspondence.  Help guide and educate customers about the fundamentals and benefits of consumer-driven health care topics to include managing their health and well-being by selecting the best benefit plan options, maximizing the value of their health plan benefits and choosing a quality care provider   Intervene with care providers (doctor's offices) on behalf of the customer to assist with appointment scheduling or connections with internal specialists for assistance when needed  Assist customers in navigating myuhc.com and other UnitedHealth Group websites and encourage and reassure them to become self-sufficient  Own problem through to resolution on behalf of the customer in real time or through comprehensive and timely follow-up with the member  Research complex issues across multiple databases and work with support resources to resolve customer issues and\/or partner with others to resolve escalated issues  Provide education and status on previously submitted pre-authorizations or pre-determination requests  Meet the performance goals established for the position in the areas of: efficiency, call quality, customer satisfaction, first call resolution and attendance          06\/2015   to   10\/2016     CSR II    Company Name   －   City  ,   State      Received an award of excellent customer service\nReceived \"Year of service Award\"\nReceived 5 year award\nReceived employee of the month twice with Brookdale Senior Living\nReceived \"Consultant Sales Award\"\nReceived employee recognition 6 months in a row along with a trophy of excellence and 6 other trophies with JP Morgan Chase.  I was a Top Performer for 2014.         10\/2013   to   06\/2015     Business Banker    Company Name   －   City  ,   State      As a Business Banker I provided guidance and assistance to business owners in managing their accounts.  I manage quality assurance, problem resolution, and a lot of data entry.  Worked a lot with numbers.  I am a Peer Coach in which I help train, guide, and motivate new hires.  Responsible for ensuring the client experience is world class and that interactions\/issues are resolved accurately and expediently.  Handle client email\/phone call interactions of a maintenance nature by retaining a thorough knowledge of the Policies and Procedures and all systems in order to assist the client and manage escalated issues.  Use a high level of client service skills in order to defuse potentially difficult and\/or upset customers with complex issues.            Took the initiative to create escalation of any issue that cannot be resolved within the anticipated service level or one that creates risk or exposure for the customer or the bank.  Identify trends within the escalated emails\/issues that lead to improvement opportunities in policies\/procedures processes.  Utilize time at work efficiently to meet or exceed production goals as outlined in the metrics scorecard.  Consistently perform follow up to ensure customer satisfaction and to validate the accuracy of work.  Was required to assist department in meeting and exceeding SLA's by handling other functions when volumes fluctuate.  Research and resolve complex client's issues by working in partnership with partner groups.  Types of cases included fraud resolution, treasury solutions, complex account maintenance, internal relationship maintenance and billing inquiries.  Support field Business Relationship Managers in the successful servicing of customers.  Investigate electronic transactions for successful resolution.  Use multiple systems and internal resources to resolve issues.  Increase customer satisfaction through excellent communication and follow up.  Document issues and record resolutions for the firm's system of record.  Acted independently within policy to assist customers and internal partners and escalate in cases where the policy is preventing us from delivering the right client experience.  Demonstrate sound decision making skills to protect the firm's interest while providing a superior customer experience.         04\/2004   to   09\/2013     Life Enrichment Coordinator\/Marketing Coordinator    Company Name  －   City  ,   State      Designed a creative and exciting life enrichment program to meet the individual needs and interests of the community.  Plan monthly calendar for the community.  Participated in discharge planning with the other members of the management team.  Coordinated the transportation, monitored budget, equipment, and supplies through Microsoft Excel.  Provided guidance and acted as a resource to staff on the importance of life enrichment.  Ensured that programs met all state, federal, local regulations.  Coordinate quarterly events at the community.  Developed and maintained contact with community agencies and supervise volunteers.  Coordinated move-ins and move-outs.  Coordinated the Welcome Committee.  Lots of Data Entry through Microsoft Excel and filing.          Education and Training     2018       Business Administration Human Resources    Argosy University          Business Administration Human Resources       2001       Paralegal Studies    Penn Foster University          Paralegal Studies       1997           Cosmetology College\n          National Beauty College                 1996     High School Diploma  :   Business and Cosmetology    W.W. Samuell High School          Business and Cosmetology        Skills    adding machines, Balance, billing, bonds, budget, Business Administration, calculators, cashier, clerical, Coach, excellent communication, Consultant, client, customer satisfaction, excellent customer service, Data Entry, decision making, discharge planning, email, filing, financial, funds, Human Resource, managing, Microsoft Excel, exchange, mail, money, Microsoft Office, Microsoft Outlook, Microsoft Power Point, Microsoft Word, photography, Policies, problem resolution, processes, purchasing, quality assurance, receiving, maintain records, Research, Sales, SLA, sound, phone, transportation, treasury, type, typing, written   """
test_normalization(sample_resume)